In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_rotten_tomatoes(base_site):
    response = requests.get(base_site)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    divs = soup.find_all('div', {'class':'col-sm-18 col-full-xs countdown-item-content'})
    heading = [div.find('h2') for div in divs]
    movie_name = [name.find('a').string for name in heading]
    years = [name.find('span', {"class":"subtle start-year"}).string for name in heading]
    clean_years = [yr.strip('()') for yr in years]
    clean_years = [int(yr) for yr in clean_years]
    scores = [head.find('span', {"class":"tMeterScore"}).string for head in heading]
    scores = [sc.strip('%') for sc in scores]
    scores = [int(sc) for sc in scores]
    consensus = [div.find('div', {'class':'info critics-consensus'}) for div in divs]
    consensus_text = [con.contents[1].strip() for con in consensus]
    director = [div.find('div', {"class":"info director"}) for div in divs]
    director_name  = [None if director.find('a') is None else director.find('a').string for director in director] 
    cast_info = [div.find('div', {'class':'info cast'}) for div in divs]
    cast = [", ".join([links.string for links in c.find_all('a')]) for c in cast_info]
    synopsis = [syn.find('div', {"class":"info synopsis"}) for syn in divs]
    synopsis_info = [syn.contents[1].strip() for syn in synopsis]
    adj_score = [div.find('div', {'class':'info countdown-adjusted-score'}) for div in divs]
    adj_score = [score.contents[1].strip('% ') for score in adj_score]

    movies_info = pd.DataFrame()
    movies_info["Title"] = movie_name
    movies_info['Released Year'] = clean_years
    movies_info['Scores'] = scores
    movies_info['Adjusted Score'] = adj_score
    movies_info['Info'] = consensus_text
    movies_info['Synopsis'] = synopsis_info
    movies_info['Director'] = director_name
    movies_info['Cast'] = cast

    return movies_info

# List of URLs to scrape
websites = ["https://editorial.rottentomatoes.com/guide/essential-movies-to-watch-now/", 
            "https://editorial.rottentomatoes.com/guide/essential-movies-to-watch-now/2/",
            "https://editorial.rottentomatoes.com/guide/best-free-youtube-movies/",
            "https://editorial.rottentomatoes.com/guide/best-horror-movies-of-all-time/",
            "https://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/"
            # you can add more URLs here
           ]

# Scrape data from each website and concatenate the results
all_movies_info = pd.concat([scrape_rotten_tomatoes(url) for url in websites], ignore_index=True)

# Exporting into a single CSV
all_movies_info.to_csv('All_Movie_Data.csv', header=True, index=False)

# Display top 10 movies by score
all_movies_info.sort_values(by="Scores", ascending=False).head(10)


,Title,Released Year,Scores,Adjusted Score,Info,Synopsis,Director,Cast
300,The Grapes of Wrath,1940,100,106444,A potent drama that is as socially important t...,"The Joad clan, introduced to the world in John...",John Ford,"Henry Fonda, Jane Darwell, John Carradine, Cha..."
711,Fist of Legend,1994,100,82395,No consensus yet.,Chinese martial arts student Chen Zhen (Jet Li...,Gordon Chan,"Jet Li, Chin Siu Ho, Billy Chow Bei-Lei, Yasua..."
407,One Cut of the Dead,2017,100,104657,"Brainy and bloody in equal measure,",Real zombies attack a hack director and a film...,Shinichiro Ueda,"Takayuki Hamatsu, Yuzuki Akiyama, Harumi Shuha..."
199,Only Yesterday,1991,100,104902,,A put-upon 27-year-old Japanese office worker ...,Isao Takahata,"Daisy Ridley, Dev Patel, Ashley Eckstein, Alis..."
427,His House,2020,100,106633,"Featuring genuine scares through every corridor,",A refugee couple makes a harrowing escape from...,Remi Weekes,"Wunmi Mosaku, Sope Dirisu, Matt Smith, Cornell..."
718,The Terminator,1984,100,106142,"With its impressive action sequences, taut eco...","Disguised as a human, a cyborg assassin known ...",James Cameron,"Arnold Schwarzenegger, Linda Hamilton, Michael..."
266,The Tale of the Princess Kaguya,2013,100,103909,"Boasting narrative depth, frank honesty, and e...",A tiny nymph found inside a bamboo stalk grows...,Isao Takahata,"Chloë Grace Moretz, James Caan, Mary Steenburg..."
120,The Grapes of Wrath,1940,100,106444,A potent drama that is as socially important t...,"The Joad clan, introduced to the world in John...",John Ford,"Henry Fonda, Jane Darwell, John Carradine, Cha..."
208,Pinocchio,1940,100,108327,"Ambitious, adventurous, and sometimes frighten...",When the woodworker Geppetto (Christian Rub) s...,Ben Sharpsteen,"Don Brodie, Walter Catlett, Frankie Darro, Cli..."
333,My Name Is Nobody,1973,100,69730,No consensus yet.,An anonymous gunfighter (Terence Hill) follows...,Tonino Valerii,"Henry Fonda, Terence Hill, Jean Martin, R. G. ..."
